In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
def supercap_discharge(k_value, label_suffix):
    t_start = 0.05  # Time at which constant discharge current begins (in seconds)
    t_total = 20  # Total duration of the test (in seconds)
    I_discharge = 2.0  # Discharge current in Amperes
    V_initial = 3.0  # Initial voltage of the supercapacitor in Volts
    k = k_value  # Capacitance factor
    c0 = 0.1  # Base capacitance
    ESR = 0.010  # Equivalent series resistance in Ohms
    step_size = 0.0001  # Step size of 0.0001 for higher resolution

    # Time axis
    t = np.arange(0, t_total, step_size)  # Fixed step size
    V = np.zeros_like(t)
    C = np.zeros_like(t)
    V_no_esr = np.zeros_like(t)  # Voltage before ESR effect

    # Compute voltage curve
    for i in range(len(t)):
        if i == 0:
            V[i] = V_initial
            V_no_esr[i] = V_initial
        elif t[i] < t_start:
            V[i] = V_initial  # Voltage remains constant before discharge
            V_no_esr[i] = V_initial
        elif t[i-1] < t_start <= t[i]:  # Apply sudden ESR drop at the moment discharge starts
            V_drop = I_discharge * ESR  # Instantaneous voltage drop due to ESR
            V_no_esr[i] = V[i-1]  # Maintain voltage before applying current discharge
            V[i] = V_no_esr[i]  # Keep the voltage before the drop
            
            # Add explicit step after 10ms to show the voltage drop
            V_no_esr[i+1] = V_no_esr[i]  # Maintain pre-drop voltage for one step
            V[i+1] = V_no_esr[i+1] - V_drop  # Apply ESR drop after one step
        
        elif t[i-1] < t_start + step_size <= t[i]:  # Ensure transition into discharge
            V_no_esr[i] = V_no_esr[i-1]  # Maintain pre-drop voltage
            V[i] = V[i-1]  # Maintain post-drop voltage
        else:
            C[i] = V_no_esr[i-1] * k + c0  # Capacitance based on voltage before ESR drop
            V_drop = I_discharge * ESR  # Voltage drop due to ESR
            new_voltage_no_esr = max(0, V_no_esr[i-1] - (I_discharge / C[i]) * step_size)
            new_voltage = max(0, new_voltage_no_esr - V_drop)  # Apply ESR drop to final voltage
            
            if new_voltage_no_esr <= 0:
                break  # Stop simulation if voltage reaches zero or below
            
            V_no_esr[i] = new_voltage_no_esr
            V[i] = new_voltage

    # Trim unused values
    t = t[:i]
    V = V[:i]
    V_no_esr = V_no_esr[:i]

    return t, V, V_no_esr, label_suffix

# Run simulations for both cases
t1, V1, V_no_esr1, label1 = supercap_discharge(0.1, 'k=0.1')
t2, V2, V_no_esr2, label2 = supercap_discharge(0.0, 'k=0.0')

# Identify points closest to 80% and 40% of V_initial for the blue line
V_80 = 0.95 * V_initial
V_40 = 0.90 * V_initial
idx_80 = (np.abs(V1 - V_80)).argmin()
idx_40 = (np.abs(V1 - V_40)).argmin()

# Use all data points between 80% and 40% voltage for the quadratic fit
t_selected = t1[idx_80:idx_40+1]
V_selected = V1[idx_80:idx_40+1]

# Fit a quadratic function using least squares over the selected range
coeffs = np.polyfit(t_selected, V_selected, 2)
quadratic_fit = np.poly1d(coeffs)
V_quad_fit = quadratic_fit(t_selected)
V_quad_fit = np.clip(V_quad_fit, 0, V_initial)

# Extend quadratic fit beyond selected points to the discharge start time
t_quad_extended = np.linspace(t_start, t1[idx_40] + (t1[idx_40] - t1[idx_80]), 200)
V_quad_extended = quadratic_fit(t_quad_extended)
V_quad_extended = np.clip(V_quad_extended, 0, V_initial)

# Compute linear regression based on the two selected points
t_linear_extended = np.linspace(t_start, t1[idx_40] + (t1[idx_40] - t1[idx_80]), 200)
slope, intercept = np.polyfit([t1[idx_80], t1[idx_40]], [V1[idx_80], V1[idx_40]], 1)
V_linear_extended = slope * t_linear_extended + intercept
V_linear_extended = np.clip(V_linear_extended, 0, V_initial)

# Plot results and save as SVG
plt.figure(figsize=(15, 6), dpi=300)
plt.plot(t1, V1, label=f'Voltage with ESR ({label1})', color='b', linewidth=2)
plt.plot(t1, V_no_esr1, label=f'Voltage before ESR drop ({label1})', linestyle='dashed', color='g'3, linewidth=1)
plt.plot(t2, V2, label=f'Voltage with ESR ({label2})', color='r', linewidth=2)
plt.plot(t2, V_no_esr2, label=f'Voltage before ESR drop ({label2})', linestyle='dashed', color='orange', linewidth=1)
plt.plot(t_linear_extended, V_linear_extended, label='Extended Linear Fit', linestyle='dotted', color='black', linewidth=2)
plt.plot(t_quad_extended, V_quad_extended, label='Extended Quadratic Fit (Least Squares)', linestyle='dashed', color='purple', linewidth=2)
plt.axvline(x=t_start, linestyle='--', color='r', label='Discharge start', linewidth=1)
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.title('Supercap Discharge with Constant Current and ESR (Comparison)')
plt.legend()
plt.grid()
plt.savefig('supercap_discharge_comparison.svg', format='svg')
plt.show()

# Print quadratic fit coefficients
a, b, c = coeffs
print(f'Quadratic Fit: V = {a:.6f}*t^2 + {b:.6f}*t + {c:.6f}')



NameError: name 'V_initial' is not defined